# Agent Loan Decisioning

In [104]:
import sys
import datetime
from datetime import date,timedelta
import numpy as np
import pandas as pd
import camelot
import matplotlib.pyplot as plt

In [105]:
# sys.maxsize
# 2147483647



Qualifying Matrix

1.	Agent Length of Service - 3 months

2.	Agent Frequency of use/activity - >= 65 transactions(count) in total

3.	Agent Frequency of rebalance - >= 45 inbound(count) in total

4.	Median Cash-In for last 90 days - avg. txn amnt - ghs5000 

5.	Mode Cash-In for last 90 days - lowest amnt cash in - ghs1000 

6.	Median Cash-Out for last 90 days - avg. txn amnt - ghs5000

7.	Mode Cash-Out for last 90 days - lowest amnt cash out - ghs1000 

In [106]:

##### full pdf reading

# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatementReport.pdf',pages='all') #emma data
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\MomoStatement2.pdf',pages='all') #prince momo agent2
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\momopages2.pdf',pages='all') # 2 pages prince
# tables = camelot.read_pdf(r'C:\Users\Chakdahah\Momopages3.pdf',pages='all') #50 pages prince
tables = camelot.read_pdf(r'C:\Users\Chakdahah\lastlast.pdf',pages='all') # 5 pages prince containig each month

tables



# tables[j].parsing_report #check parsing report to see accuracy of parsing and other details

# append to one csv file

table_fin = tables[0].df #convert first table to df and assign to variable 

for i in range(1,len(tables)-1): # skip 0-nth of tables
    print(i)
    t = tables[i].df 
    t.drop(t.index[0], inplace = True) # drop first row of df
    table_fin = pd.concat([table_fin, t], axis = 0)
    

    print(t)


1
                                                   0                       1   \
1                             31-Mar-2023 01:47:07 PM                65223666   
2                             31-Mar-2023 01:42:22 PM                65223666   
3   31-Mar-2023 01:34:52 PM\nTRANSACTION DATE\n31-...  0\nFROM ACCT\n60384156   
4                                                                               
5                                                                               
6                             31-Mar-2023 01:29:48 PM                       0   
7                             31-Mar-2023 01:29:48 PM                52266321   
8                             31-Mar-2023 01:27:17 PM                65223666   
9                             31-Mar-2023 01:22:29 PM                       0   
10                            31-Mar-2023 01:22:29 PM                50002947   
11                            31-Mar-2023 01:07:04 PM                65223666   
12                        

In [107]:
# Assign tables into momo data variable

momo_data = table_fin
momo_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
1,31-Mar-2023 07:48:12 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,210,0,1764.24,1554.24,233244681179,Teddy Pendeglass Kissi,47508410,23222467375,Cashin To-\n233244681179,Internal
2,31-Mar-2023 07:47:49 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,200,0,1964.24,1764.24,233548054905,KISSI TEDDY,66016708,23222456200,Cashin To-\n233548054905,Internal
3,31-Mar-2023 07:39:32 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,30,0,1994.24,1964.24,233542482423,KELVIN APPIAH,71255668,23222206348,Cashin To-\n233542482423,Internal
4,31-Mar-2023 07:09:58 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,110,0,2104.24,1994.24,233540590150,GIDEON ATIAH,72580914,23221272044,Cashin To-\n233540590150,Internal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,28-Jan-2023 08:39:39 AM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,25.5,0,522.2,496.7,233541403149,Elizabeth Dzoka,53878073,21745041950,Cashin To-\n233541403149,Internal
36,28-Jan-2023 08:39:06 AM,0,,0,COMMISSIONING,1.14,0,235.6,236.74,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,,Internal
37,28-Jan-2023 08:39:06 AM,41861402,Rebecca Caro,233243931345,CASH_OUT,285,2.85,237.2,522.2,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,NationalId--,Internal
38,28-Jan-2023 08:33:21 AM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,454,0,691.2,237.2,233256519321,DORIS NIMAKO,69336653,21744920822,Cashin To-\n233256519321,Internal


In [108]:
momo_data[~momo_data.iloc[:,0].str.contains("\n")]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
1,31-Mar-2023 07:48:12 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,210,0,1764.24,1554.24,233244681179,Teddy Pendeglass Kissi,47508410,23222467375,Cashin To-\n233244681179,Internal
2,31-Mar-2023 07:47:49 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,200,0,1964.24,1764.24,233548054905,KISSI TEDDY,66016708,23222456200,Cashin To-\n233548054905,Internal
3,31-Mar-2023 07:39:32 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,30,0,1994.24,1964.24,233542482423,KELVIN APPIAH,71255668,23222206348,Cashin To-\n233542482423,Internal
4,31-Mar-2023 07:09:58 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,110,0,2104.24,1994.24,233540590150,GIDEON ATIAH,72580914,23221272044,Cashin To-\n233540590150,Internal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,28-Jan-2023 08:39:39 AM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,25.5,0,522.2,496.7,233541403149,Elizabeth Dzoka,53878073,21745041950,Cashin To-\n233541403149,Internal
36,28-Jan-2023 08:39:06 AM,0,,0,COMMISSIONING,1.14,0,235.6,236.74,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,,Internal
37,28-Jan-2023 08:39:06 AM,41861402,Rebecca Caro,233243931345,CASH_OUT,285,2.85,237.2,522.2,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,NationalId--,Internal
38,28-Jan-2023 08:33:21 AM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,454,0,691.2,237.2,233256519321,DORIS NIMAKO,69336653,21744920822,Cashin To-\n233256519321,Internal


In [109]:
momo_data.columns = momo_data.iloc[0] #set column names equal to values in row index position 0
momo_data = momo_data[1:] # remove first row from DataFrame

In [110]:
momo_data.reset_index(drop=True, inplace=True)

In [111]:
momo_data

,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
0,31-Mar-2023 07:48:12 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,210,0,1764.24,1554.24,233244681179,Teddy Pendeglass Kissi,47508410,23222467375,Cashin To-\n233244681179,Internal
1,31-Mar-2023 07:47:49 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,200,0,1964.24,1764.24,233548054905,KISSI TEDDY,66016708,23222456200,Cashin To-\n233548054905,Internal
2,31-Mar-2023 07:39:32 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,30,0,1994.24,1964.24,233542482423,KELVIN APPIAH,71255668,23222206348,Cashin To-\n233542482423,Internal
3,31-Mar-2023 07:09:58 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,110,0,2104.24,1994.24,233540590150,GIDEON ATIAH,72580914,23221272044,Cashin To-\n233540590150,Internal
4,31-Mar-2023 07:01:26 PM,65223635,MAXWELL FOSU\nNYARKOH,233241435761,TRANSFER,200,0,1904.24,2104.24,233591451241,MAXWELL FOSU NYARKOH,65223666,23220963330,1,Internal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,28-Jan-2023 08:39:39 AM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,25.5,0,522.2,496.7,233541403149,Elizabeth Dzoka,53878073,21745041950,Cashin To-\n233541403149,Internal
126,28-Jan-2023 08:39:06 AM,0,,0,COMMISSIONING,1.14,0,235.6,236.74,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,,Internal
127,28-Jan-2023 08:39:06 AM,41861402,Rebecca Caro,233243931345,CASH_OUT,285,2.85,237.2,522.2,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,NationalId--,Internal
128,28-Jan-2023 08:33:21 AM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,454,0,691.2,237.2,233256519321,DORIS NIMAKO,69336653,21744920822,Cashin To-\n233256519321,Internal


In [112]:
#Reading data from csv file

# momo_data = pd.read_csv(r'\Users\Chakdahah\Cleaned_Momo.csv', skipinitialspace = True)

# Clean whitespaces in colunn name and entire dataframe

momo_data =momo_data.rename(columns=lambda x: x.strip())
momo_data =momo_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

momo_data.head()

,TRANSACTION DATE,FROM ACCT,FROM NAME,FROM NO.,TRANS. TYPE,AMOUNT,FEES,BAL BEFORE,BAL AFTER,TO NO.,TO NAME,TO ACCT,F_ID,REF,OVA
0,31-Mar-2023 07:48:12 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,210,0,1764.24,1554.24,233244681179,Teddy Pendeglass Kissi,47508410,23222467375,Cashin To-\n233244681179,Internal
1,31-Mar-2023 07:47:49 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,200,0,1964.24,1764.24,233548054905,KISSI TEDDY,66016708,23222456200,Cashin To-\n233548054905,Internal
2,31-Mar-2023 07:39:32 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,30,0,1994.24,1964.24,233542482423,KELVIN APPIAH,71255668,23222206348,Cashin To-\n233542482423,Internal
3,31-Mar-2023 07:09:58 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,110,0,2104.24,1994.24,233540590150,GIDEON ATIAH,72580914,23221272044,Cashin To-\n233540590150,Internal
4,31-Mar-2023 07:01:26 PM,65223635,MAXWELL FOSU\nNYARKOH,233241435761,TRANSFER,200,0,1904.24,2104.24,233591451241,MAXWELL FOSU NYARKOH,65223666,23220963330,1,Internal


In [113]:

# remove unwanted characters from end of column names
# mmomo_data.rename(columns=lambda x: x.strip('.'),inplace =True)

# remove all unwanted char acters in column names
momo_data.rename(columns=lambda s: s.replace(".", ""), inplace=True)

# replace whitespaces with underscore
momo_data.rename(columns=lambda s: s.replace(" ", "_"), inplace=True)

In [114]:
# cast appropriate data types
momo_data['AMOUNT'] = pd.to_numeric(momo_data['AMOUNT'],errors='coerce')
momo_data['BAL_BEFORE'] = pd.to_numeric(momo_data['BAL_BEFORE'],errors='coerce')
momo_data['BAL_AFTER'] = pd.to_numeric(momo_data['BAL_AFTER'],errors='coerce')
momo_data['F_ID'] = pd.to_numeric(momo_data['F_ID'],errors='coerce')


In [115]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TRANSACTION_DATE  130 non-null    object 
 1   FROM_ACCT         130 non-null    object 
 2   FROM_NAME         130 non-null    object 
 3   FROM_NO           130 non-null    object 
 4   TRANS_TYPE        130 non-null    object 
 5   AMOUNT            125 non-null    float64
 6   FEES              130 non-null    object 
 7   BAL_BEFORE        125 non-null    float64
 8   BAL_AFTER         125 non-null    float64
 9   TO_NO             130 non-null    object 
 10  TO_NAME           130 non-null    object 
 11  TO_ACCT           130 non-null    object 
 12  F_ID              125 non-null    float64
 13  REF               130 non-null    object 
 14  OVA               130 non-null    object 
dtypes: float64(4), object(11)
memory usage: 15.4+ KB


In [116]:
#Suppressing any scientific notation in entire csv file.

pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [117]:
momo_data.head()

,TRANSACTION_DATE,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
0,31-Mar-2023 07:48:12 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,210,0,1764,1554,233244681179,Teddy Pendeglass Kissi,47508410,23222467375,Cashin To-\n233244681179,Internal
1,31-Mar-2023 07:47:49 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,200,0,1964,1764,233548054905,KISSI TEDDY,66016708,23222456200,Cashin To-\n233548054905,Internal
2,31-Mar-2023 07:39:32 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,30,0,1994,1964,233542482423,KELVIN APPIAH,71255668,23222206348,Cashin To-\n233542482423,Internal
3,31-Mar-2023 07:09:58 PM,65223666,MAXWELL FOSU\nNYARKOH,233591451241,CASH_IN,110,0,2104,1994,233540590150,GIDEON ATIAH,72580914,23221272044,Cashin To-\n233540590150,Internal
4,31-Mar-2023 07:01:26 PM,65223635,MAXWELL FOSU\nNYARKOH,233241435761,TRANSFER,200,0,1904,2104,233591451241,MAXWELL FOSU NYARKOH,65223666,23220963330,1,Internal


In [118]:
m =momo_data['TRANSACTION_DATE'].str.split(' ', expand=True)


In [119]:

momo_data['TRANSACTION_DATE'] = m[0]
momo_data['TRANSACTION_TIME'] = m[1]


In [120]:
# shift column 'C' to first position
first_column = momo_data.pop('TRANSACTION_TIME')
  
# insert column using insert(position,column_name,first_column) function
momo_data.insert(1, 'TRANSACTION_TIME', first_column)
  

In [121]:
momo_data.info(all)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   TRANSACTION_DATE  130 non-null    object 
 1   TRANSACTION_TIME  128 non-null    object 
 2   FROM_ACCT         130 non-null    object 
 3   FROM_NAME         130 non-null    object 
 4   FROM_NO           130 non-null    object 
 5   TRANS_TYPE        130 non-null    object 
 6   AMOUNT            125 non-null    float64
 7   FEES              130 non-null    object 
 8   BAL_BEFORE        125 non-null    float64
 9   BAL_AFTER         125 non-null    float64
 10  TO_NO             130 non-null    object 
 11  TO_NAME           130 non-null    object 
 12  TO_ACCT           130 non-null    object 
 13  F_ID              125 non-null    float64
 14  REF               130 non-null    object 
 15  OVA               130 non-null    object 
dtypes: float64(4), object(12)
memory usage: 16.4

In [122]:
momo_data['FROM_NAME'] = momo_data.FROM_NAME.str.replace('\n',' ')

In [123]:
momo_data.tail()

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
125,28-Jan-2023,08:39:39,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,26,0,522,497,233541403149,Elizabeth Dzoka,53878073,21745041950,Cashin To-\n233541403149,Internal
126,28-Jan-2023,08:39:06,0,,0,COMMISSIONING,1,0,236,237,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,,Internal
127,28-Jan-2023,08:39:06,41861402,Rebecca Caro,233243931345,CASH_OUT,285,2.85,237,522,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,NationalId--,Internal
128,28-Jan-2023,08:33:21,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,454,0,691,237,233256519321,DORIS NIMAKO,69336653,21744920822,Cashin To-\n233256519321,Internal
129,28-Jan-2023,08:28:29,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,202,0,893,691,233599835915,FAUSTINA NTORI,67458540,21744827791,Cashin To-\n233599835915,28-Jan-2023 11:10:46 AM\n0\n0\nCOMMISSIONING\n...


In [124]:
num=momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_IN', 'FROM_NO'].iloc[0]
num

'233591451241'

In [125]:
name=momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_IN', 'FROM_NAME'].iloc[0]
name

'MAXWELL FOSU NYARKOH'

In [126]:
momo_data[~momo_data.iloc[:,0].str.contains("\n")]

,TRANSACTION_DATE,TRANSACTION_TIME,FROM_ACCT,FROM_NAME,FROM_NO,TRANS_TYPE,AMOUNT,FEES,BAL_BEFORE,BAL_AFTER,TO_NO,TO_NAME,TO_ACCT,F_ID,REF,OVA
0,31-Mar-2023,07:48:12,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,210,0,1764,1554,233244681179,Teddy Pendeglass Kissi,47508410,23222467375,Cashin To-\n233244681179,Internal
1,31-Mar-2023,07:47:49,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,200,0,1964,1764,233548054905,KISSI TEDDY,66016708,23222456200,Cashin To-\n233548054905,Internal
2,31-Mar-2023,07:39:32,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,30,0,1994,1964,233542482423,KELVIN APPIAH,71255668,23222206348,Cashin To-\n233542482423,Internal
3,31-Mar-2023,07:09:58,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,110,0,2104,1994,233540590150,GIDEON ATIAH,72580914,23221272044,Cashin To-\n233540590150,Internal
4,31-Mar-2023,07:01:26,65223635,MAXWELL FOSU NYARKOH,233241435761,TRANSFER,200,0,1904,2104,233591451241,MAXWELL FOSU NYARKOH,65223666,23220963330,1,Internal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,28-Jan-2023,08:39:39,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,26,0,522,497,233541403149,Elizabeth Dzoka,53878073,21745041950,Cashin To-\n233541403149,Internal
126,28-Jan-2023,08:39:06,0,,0,COMMISSIONING,1,0,236,237,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,,Internal
127,28-Jan-2023,08:39:06,41861402,Rebecca Caro,233243931345,CASH_OUT,285,2.85,237,522,233591451241,MAXWELL FOSU NYARKOH,65223666,21745031302,NationalId--,Internal
128,28-Jan-2023,08:33:21,65223666,MAXWELL FOSU NYARKOH,233591451241,CASH_IN,454,0,691,237,233256519321,DORIS NIMAKO,69336653,21744920822,Cashin To-\n233256519321,Internal


In [127]:
df3=momo_data.loc[momo_data['TRANS_TYPE'] == 'CASH_IN', 'FROM_NAME'].iloc[0]
df3

'MAXWELL FOSU NYARKOH'

In [128]:
momo_data =momo_data.dropna()

In [129]:
# store date as object for later use
mi = momo_data['TRANSACTION_DATE']

In [130]:
momo_data['TRANSACTION_DATE'] = pd.to_datetime(momo_data['TRANSACTION_DATE'])

In [131]:
## convert momo_data to csv
momo_data.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\StrippedMomotoCSV200.csv',date_format='%d/%m/%Y')

momo_data.to_excel("StrippedMomotoXLS.xlsx")

In [132]:
# Set empty list and dict
masterData = []
userData= {}

# Calculation of scoring parameters

In [133]:

rows = momo_data.count()[0] # Agent Frequency of use
userData["freq_of_active"] = rows



MOMO=momo_data[(momo_data == 'CASH_IN').any(axis=1)] # frequency of rebalance
MOMOR = MOMO.count()[0]
userData["freq_of_repl"] = MOMOR



meadnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].median() #Median Cash-In for last 90days
# userData["mead_cash_in"] = meadnie
# userData["mead_cash_in"]
userData["mead_cash_in_calc"] = meadnie*4


modnie = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_IN','AMOUNT'].mode() # Mode Cash-In for last 90 days
userData["modnie_cash_in_calc"] = modnie*4
userData["modnie_cash_in_calc"] = userData["modnie_cash_in_calc"]*4
userData["modnie_cash_in_calc"] = max(userData["modnie_cash_in_calc"])


meadniout = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].median() # Median Cash-Out for last 90 days
# userData["mead_cash_out"] = meadniout
# userData["mead_cash_out"]
userData["mead_cash_out_calc"] = meadniout*4


modno = momo_data.loc[momo_data['TRANS_TYPE']=='CASH_OUT','AMOUNT'].mode() # Mode Cash-Out for last 90 days
userData["modno_cash_out_calc"] = modno*4 
userData["modno_cash_out_calc"] = userData["modno_cash_out_calc"]*4
userData["modno_cash_out_calc"] = max(userData["modno_cash_out_calc"])




# Scoring Criteria Calculation

In [134]:
user_copy = userData.copy()
masterData.append(user_copy)

In [135]:
### scoring

inelligible = []
elligible = []
score_table = []
scoring_table = {}

In [136]:
scoring_table["id Number"] = num
scoring_table["Agent Name"] = name

In [137]:


## Agent Length of Service - 3 months

for i in mi:
    u = mi.str.extract('([a-zA-Z ]+)', expand=False).str.strip()
    monthcount = u.drop_duplicates()
print(monthcount)

if len(monthcount)>=3:
    scoring_table["Length of Service"] = 100
    Credit_score = 100
else:
    scoring_table["Length of Service"] = 0
    Credit_score = 0


for index in range(len(masterData)):

    ## Frequency of use  >= 65 transactions(count) in total

    if masterData[index]["freq_of_active"]> 65 and masterData[index]["freq_of_active"] <= 75:
        user_copy = masterData[index]
#         print(user_copy)
        Credit_score += 50
        scoring_table["Frequency of Activity"] = 50
    elif masterData[index]["freq_of_active"]> 75:
        elig_copy = masterData[index]
        Credit_score+= 100
        scoring_table["Frequency of Activity"] = 100
    else:
        scoring_table["Frequency of Activity"] = 0


        

        
        
#Agent Frequency of rebalance - >= 45 inbound(count) in total

    if masterData[index]["freq_of_repl"]>= 45 and masterData[index]["freq_of_repl"] <= 54:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Frequency of Rebalance"] = 50
    elif masterData[index]["freq_of_active"]> 54:
        elig_copy = masterData[index]
        Credit_score+= 100
        scoring_table["Frequency of Rebalance"] = 100
    else:
        scoring_table["Frequency of Rebalance"] = 0
        

        
        
        # Median Cash-In for last 90 days - avg. txn amnt - ghs5000 

        
    if masterData[index]["mead_cash_in_calc"] >= 5000:
        user_copy = masterData[index]
        Credit_score += 50
        scoring_table["Median Cash In"] = 50
    else:
        Credit_score+= 0
        scoring_table["Median Cash In"] = 0

        
        
          # Mode Cash-In for last 90 days - lowest amnt cash in - ghs1000 

        
    if masterData[index]["modnie_cash_in_calc"] >= 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash In"] = 50
        
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        scoring_table["Mode Cash In"] = 0
        
        
        
       # Median Cash-Out for last 90 days - avg. txn amnt - ghs5000


    if masterData[index]["mead_cash_out_calc"] > 5000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Median Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score+= 0
        scoring_table["Median Cash Out"] = 0
        
#     Mode Cash-Out for last 90 days - lowest amnt cash out - ghs1000 
        
    if masterData[index]["modno_cash_out_calc"] >= 1000:
        user_copy = masterData[index]
        Credit_score += 50 
        scoring_table["Mode Cash Out"] = 50
    else:
        elig_copy = masterData[index]
        Credit_score+= 0   
        scoring_table["Mode Cash Out"]
    

#         print(elig_copy)
scoring_table["Credit Score"] = Credit_score
score_table.append(scoring_table)        
        
    

0     Mar
61    Feb
91    Jan
Name: TRANSACTION_DATE, dtype: object


# Applying scores to Merchants

In [138]:
    
    if Credit_score >= 200 and Credit_score <= 250:
        print("You qualify for a GHS 1000 loan")
        scoring_table["Eligibility"] = 1000

    elif Credit_score >= 251 and Credit_score <= 350:
        print("You qualify for a GHS 2000 loan")
        scoring_table["Eligibility"] = 2000

    elif Credit_score >= 351 and Credit_score <= 400:
        print("You qualify for a GHS 3000 loan")
        scoring_table["Eligibility"] = 3000


    elif Credit_score >= 401 and Credit_score <= 500:
        print("You qualify for a GHS 5000 loan")
        scoring_table["Eligibility"] = 5000



You qualify for a GHS 3000 loan


In [139]:
elligible.append(elig_copy)

In [140]:
print("Elligible", elligible)

Elligible [{'freq_of_active': 125, 'freq_of_repl': 55, 'mead_cash_in_calc': 1010.0, 'modnie_cash_in_calc': 1616.0, 'mead_cash_out_calc': 780.0, 'modno_cash_out_calc': 1600.0}]


In [141]:
Credit_score

400

In [142]:
elligible

[{'freq_of_active': 125,
  'freq_of_repl': 55,
  'mead_cash_in_calc': 1010.0,
  'modnie_cash_in_calc': 1616.0,
  'mead_cash_out_calc': 780.0,
  'modno_cash_out_calc': 1600.0}]

In [143]:
type(elligible)

list

In [144]:
score_table = pd.DataFrame.from_dict(score_table)
score_table

,id Number,Agent Name,Length of Service,Frequency of Activity,Frequency of Rebalance,Median Cash In,Mode Cash In,Median Cash Out,Mode Cash Out,Credit Score,Eligibility
0,233591451241,MAXWELL FOSU NYARKOH,100,100,100,0,50,0,50,400,3000


In [145]:
score_table.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\scoretable.csv')

In [146]:
### convert dict to df

elligible = pd.DataFrame.from_dict(elligible)
elligible

,freq_of_active,freq_of_repl,mead_cash_in_calc,modnie_cash_in_calc,mead_cash_out_calc,modno_cash_out_calc
0,125,55,1010,1616,780,1600


In [147]:
type(elligible)

pandas.core.frame.DataFrame

In [148]:
elligible.to_csv(r'C:\Users\Chakdahah\Downloads\Django-main\Django-main\analytics_project\elligible.csv')

# Score and risk grade table 

In [149]:
details = {
    'Risk Ranking' : ['RR-1', 'RR-2', 'RR-3', 'RR-4'],
    'Minimum Score' : [200,251,351,401],
    'Maximum Score' : [250,350,400,500],
    'Qualifying Amount(GHS)':[1000,2000,3000,5000]
}
  
# creating a Dataframe object 
df = pd.DataFrame(details)
df

,Risk Ranking,Minimum Score,Maximum Score,Qualifying Amount(GHS)
0,RR-1,200,250,1000
1,RR-2,251,350,2000
2,RR-3,351,400,3000
3,RR-4,401,500,5000
